# Bar Graph Detection

In [1]:
from opencv_alvin import *
import pytesseract as pyt
%matplotlib inline

In [167]:
#import pytesseract as pyt
def remove_text(img):
    h=img.shape[0]
    boxes=pyt.image_to_boxes(img)
    for b in boxes.splitlines():
        b = b.split(' ')
        charLen_x=int((int(b[3])-int(b[1]))*0.5)
        charLen_y=int((int(b[4])-int(b[2]))*0.5)
        if charLen_x>img.shape[0]*0.3 or charLen_y>img.shape[1]*0.3:
            continue
        else:
            img = cv2.rectangle(img, (int(b[1])-charLen_x,h - int(b[2])+charLen_y), (int(b[3])+charLen_x, h - int(b[4])-charLen_y), (255, 255, 255), -1)
    im(img,"after removing the text")
    return img

def bar_graph_detector(img_org,IM):
    img=cv2.GaussianBlur(img_org,(3,3),0)
    im(img,"gaussian blur")
    ''' img should be grayscale and enhanced without text'''
    kernel=np.ones((5,5),np.uint8)
    img=cv2.morphologyEx(img,cv2.MORPH_CROSS,kernel,iterations=2)
    #img=cv2.dilate()
    #im(img,"before removing text")
    #img=remove_text(img)
    im(img,"after morphology")
    #cv2.equalizeHist(img,img)
    #im(img,"equalize hist")
    #img=cv2.bilateralFilter(img,9,75, 75)
    #im(img,"bilateral filter")
    v = np.median(img_org)
    sigma=0.33
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    canny_edges = cv2.Canny(img, lower, upper)
    #canny_edges=cv2.Canny(img,20,100)
    im(canny_edges,'canny images')
    ci,contour,hierarchy=cv2.findContours(canny_edges,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    count=0
    disp=IM
    tot_len=len(contour)
    if tot_len<2:
        return (0,0,0)
    for c in contour:

#         accuracy=0.001*cv2.arcLength(c,True)
        #approx=cv2.approxPolyDP(c,accuracy,True)
        hull=cv2.convexHull(c)
        approx=hull
        if 4<=len(approx)<=7:
            count+=1
            disp=cv2.drawContours(disp,[c],0,(0,255,0),2)
        else:
            disp=cv2.drawContours(disp,[c],0,(0,0,255),2)
            #print(len(approx))
    #print(str("accuracy is : ")+str(count/tot_len)+str(" ")+str(tot_len))
    return (disp,count,tot_len)

In [173]:
IM=cv2.imread('bar_graph_png/10.png')
IM_G=cv2.cvtColor(IM,cv2.COLOR_BGR2GRAY)
im(IM)


In [68]:
string=pyt.image_to_string(IM)
string

''

In [147]:
def res_generator(IM,IM_G):
    res_img,success,total=bar_graph_detector(IM_G,IM)
    if total==0:
        return (False,0)
    else:
        accuracy=success/total
        im(res_img)
        if success/total>=0.5:
            print("its a bar graph "+str(accuracy))
            #return (True,accuracy)
        else:
            print("Not a bar graph "+str(accuracy))
            #return (False,accuracy)
#im(res_img)


In [169]:

img = cv2.imread('line_chart/1.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

v = np.median(gray)
sigma=0.80
lower = int(max(0, (1.0 - sigma) * v))
upper = int(min(255, (1.0 + sigma) * v))
    
    
edges = cv2.Canny(gray,lower,upper,apertureSize = 3)
im(edges)
minLineLength = 2
maxLineGap = 1
lines = cv2.HoughLinesP(edges,1,np.pi/180,10,minLineLength,maxLineGap)
for j in lines:
    for x1,y1,x2,y2 in j:
        cv2.line(img,(x1,y1),(x2,y2),(0,255,0),5)
im(img,'houghlines5.jpg')
len(lines)


error: OpenCV(3.4.1) /io/opencv/modules/imgproc/src/color.cpp:11147: error: (-215) scn == 3 || scn == 4 in function cvtColor


In [174]:
res_generator(IM,IM_G)

its a bar graph 0.75


## loop for all images


In [4]:
li=[]
for i in range(1,21):
    name_generated="bar_graph_png/"+str(i)+".png"
    print(name_generated)
    main_image=cv2.imread(str(name_generated))
    #im(main_image)
    IM_G=cv2.cvtColor(main_image,cv2.COLOR_BGR2GRAY)
    li.append(res_generator(main_image,IM_G))
    


bar_graph_png/1.png
bar_graph_png/2.png
bar_graph_png/3.png
bar_graph_png/4.png
bar_graph_png/5.png
bar_graph_png/6.png
bar_graph_png/7.png
bar_graph_png/8.png
bar_graph_png/9.png
bar_graph_png/10.png
bar_graph_png/11.png
bar_graph_png/12.png
bar_graph_png/13.png
bar_graph_png/14.png
bar_graph_png/15.png
bar_graph_png/16.png
bar_graph_png/17.png
bar_graph_png/18.png
bar_graph_png/19.png
bar_graph_png/20.png


In [11]:
for (number,i) in enumerate(li):
    if i[0]==False:
        print(str(number+1)+str(i))

2(False, 0.3333333333333333)
4(False, 0.5)
10(False, 0.4)
13(False, 0.40816326530612246)
16(False, 0.3333333333333333)
20(False, 0.42857142857142855)
